# Spectral Clustering
Spectral clustering is a powerful and versatile clustering method that is based on the principles of graph theory and linear algebra. Instead of directly clustering the data in the input space, it constructs a similarity graph where data points are nodes and edges represents similarities between the points.

The algorithm then leverages the spectral properties of the graph (i.e., the eigenvalues and eigenvectors of the graph’s Laplacian matrix) to project the data into a lower-dimensional space. In the transformed space, traditional clustering techniques, such as k-means, can be applied more effectively, even when the data has complex, non-linear boundaries in the original space.

## Similarity Graph
The first step in spectral clustering is to represent the data as a graph, where each node corresponds to a data point, and edges connect nodes based on the distance (or similarity) between the points.

There are three common ways to construct the similarity graph:

1. **Fully connected:** Every node is connected to every other node, and the edges of the graph are weighted by the similarity of their endpoints.
2. **k-nearest neighbors:** Each node is connected only to its k nearest nodes.
3. **$\epsilon$-neighborhood:** Nodes are connected only if their distance is less than $\epsilon$.

In the case of a fully connected graph, the weighted adjacency matrix (or adjacency matrix, for short) $W$ of the graph is defined as follows:

$$W_{ij} = \begin{cases}
    s(x_i, x_j), \quad i \neq j \\
    0, \quad\quad i = j
    \end{cases}
$$

where $s(x_i, x_j)$ is the the similarity between data points $x_i$ and $x_j$. A commonly used similarity function is the Gaussian radial basis function (RBF):

$$s(x_i, x_j) = e^{- \frac{|| x_i - x_j ||^2}{2 \sigma^2}}$$

where $\sigma$ is a parameter that controls the width of the neighborhood around each point.

For k-nearest neighbors and $\epsilon$-neighborhood graphs, the adjacency matrix is usually binary, i.e., $W_{ij} = 1$ if nodes $i$ and $j$ are connected, and 0 otherwise.

In [ ]:
For example, assume that we have the following dataset of 10 two-dimensional points: